In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import pytz
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.utils import resample

import google.colab.drive
google.colab.drive.mount('/content/drive')

# Function to read and clean data
def read_clean(file):
    df = pd.read_csv(file)
    # Clean column names (equivalent to janitor::clean_names)
    df.columns = [col.lower().replace(' ', '_').replace('.', '_') for col in df.columns]
    return df[['anon_student_id', 'time_3', 'detector_name', 'value']].rename(columns={'time_3': 'time'})

Mounted at /content/drive


# Don't run

In [ ]:
# Find all CSV files in directory
files = glob.glob('/Desktop/LearnSphereDataMatch/*.csv')

# Read and combine all files
d_detector_list = []
for f in files:
    d_detector_list.append(read_clean(f))

d_detector = pd.concat(d_detector_list, ignore_index=True)

# Filter out rows where detector_name is 'False'
d_detector = d_detector[d_detector['detector_name'] != 'False']

# Recode values (naive recoding)
d_detector['value'] = d_detector['value'].apply(lambda x: 0 if str(x).startswith('0') else 1)

# Group by and pivot
d_detector = d_detector.groupby(['anon_student_id', 'time', 'detector_name']).agg({'value': 'max'}).reset_index()
d_detector = d_detector.pivot_table(index=['anon_student_id', 'time'],
                                    columns='detector_name',
                                    values='value',
                                    fill_value=0).reset_index()

# Parse time to datetime
d_detector['time'] = pd.to_datetime(d_detector['time'], utc=True)
d_detector['time'] = d_detector['time'].dt.tz_localize(None)  # Remove timezone info

# Run This

In [ ]:
# Load main dataset
# d = pd.read_csv('/content/drive/MyDrive/misuse+doingwell+struggle+idle_R_analysis.csv')
#d = pd.read_csv('/content/drive/MyDrive/misuse_doingwell_struggle_idle_5_percentage_of_data.csv')
d = pd.read_csv('/content/drive/MyDrive/misuse_doingwell_struggle_idle_5_percentage_new_threshold.csv')
d.columns = [col.lower().replace(' ', '_').replace('.', '_') for col in d.columns]
d.columns

Index(['anon_student_id', 'session_id', 'time', 'kc_model(mathia)',
       'level_(section)', 'problem_name', 'step_name', 'attempt_at_step',
       'help_level', 'selection', 'action', 'input', 'outcome', 'cf_(rule_id)',
       'cf_(semantic_event_id)', 'cf_(etalon)', 'cf_(skill_previous_p-known)',
       'cf_(skill_new_p-known)', 'cf_(module)', 'cf_(workspace_encounter)',
       'cf_(workspace_variant)', 'cf_(workspace_progress_status)',
       'cf_(skill_opportunity)', 'cf_school_id', 'cf_class_id',
       'selection_<-_ifelse(action_==_"done_button",_"done_button",_na)',
       'helpedtransaction', 'system_misuse', 'student_doing_well', 'idle',
       'struggle'],
      dtype='object')

In [ ]:
d['requested_help'] = np.where(d['help_level'] > 0, 1, 0)
d['got_help'] = np.where(d['helpedtransaction'], 1, 0)

# Modeling setup
mins_agg = "15min"

# Round and group by time window
d_log = d[['anon_student_id', 'cf_class_id', 'time', 'requested_help', 'got_help']].copy()
d_log['time'] = pd.to_datetime(d_log['time']).dt.round(mins_agg)

# Group by and summarize
d_log = d_log.groupby(['anon_student_id', 'cf_class_id', 'time']).agg({
    'requested_help': lambda x: 1 if sum(x) > 0 else 0,
    'got_help': lambda x: 1 if sum(x) > 0 else 0
}).reset_index()

In [ ]:
# Group detector data
d_detector_join = d.groupby(['anon_student_id', 'time']).agg({
    #'critical_struggle': lambda x: 1 if sum(x) > 0 else 0,
    'idle': lambda x: 1 if sum(x) > 0 else 0,
    'system_misuse': lambda x: 1 if sum(x) > 0 else 0,
    'struggle': lambda x: 1 if sum(x) > 0 else 0,
    'student_doing_well': lambda x: 1 if sum(x) > 0 else 0
}).reset_index()

d_detector_join['time'] = pd.to_datetime(d_detector_join['time']).dt.round(mins_agg)


print(d_detector_join['struggle'].value_counts())
print(d_detector_join['student_doing_well'].value_counts())
print(d_detector_join['idle'].value_counts())
print(d_detector_join['system_misuse'].value_counts())

struggle
0    73132
1       17
Name: count, dtype: int64
student_doing_well
0    64905
1     8244
Name: count, dtype: int64
idle
0    72963
1      186
Name: count, dtype: int64
system_misuse
0    70202
1     2947
Name: count, dtype: int64


In [ ]:
d_model = pd.merge(d_log, d_detector_join, on=['anon_student_id', 'time'], how='left')
d_model = d_model.fillna(0)

# 1. Mixed Effects Models for Help-Seeking Behavior

In [ ]:
# Mixed Effects Modeling with statsmodels
import statsmodels.formula.api as smf

# Model 1 - Only with requested_help
# formula1 = "got_help ~ requested_help"
# model1 = smf.mixedlm(formula1, d_model, groups=d_model["anon_student_id"], re_formula="~1")
# result1 = model1.fit()
#print(result1.summary())

# Model 2 - With requested_help and idle
# formula2 = "got_help ~ requested_help + idle"
# model2 = smf.mixedlm(formula2, d_model, groups=d_model["anon_student_id"], re_formula="~1")
# result2 = model2.fit()
#print(result2.summary())

# Model 3 - With all predictors
# formula3 = "got_help ~ requested_help + idle + system_misuse + struggle + student_doing_well"
# model3 = smf.mixedlm(formula3, d_model, groups=d_model["anon_student_id"], re_formula="~1")
# result3 = model3.fit()
#print(result3.summary())

# Model 4 - Predict requested_help
formula4 = "requested_help ~ idle + system_misuse + struggle + student_doing_well"
model4 = smf.mixedlm(formula4, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result4 = model4.fit()
print(result4.summary())

formula = "requested_help ~ idle"
model = smf.mixedlm(formula, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result = model.fit()
print(result.summary())

formula = "requested_help ~ struggle"
model = smf.mixedlm(formula, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result = model.fit()
print(result.summary())

formula = "requested_help ~ idle + struggle"
model = smf.mixedlm(formula, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result = model.fit()
print(result.summary())

formula = "requested_help ~ student_doing_well"
model = smf.mixedlm(formula, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result = model.fit()
print(result.summary())

formula = "requested_help ~ system_misuse + student_doing_well"
model = smf.mixedlm(formula, d_model, groups=d_model["anon_student_id"], re_formula="~1")
result = model.fit()
print(result.summary())

            Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: requested_help
No. Observations:   73149   Method:             REML          
No. Groups:         283     Scale:              0.1854        
Min. group size:    4       Log-Likelihood:     -42732.8765   
Max. group size:    1504    Converged:          Yes           
Mean group size:    258.5                                     
--------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept           0.607    0.016  37.786 0.000  0.575  0.638
idle                0.013    0.032   0.422 0.673 -0.049  0.076
system_misuse       0.144    0.009  16.968 0.000  0.128  0.161
struggle            0.052    0.105   0.492 0.623 -0.154  0.257
student_doing_well -0.199    0.005 -37.155 0.000 -0.209 -0.188
Group Var           0.070    0.015                            

    

# 2. Identifying Challenge Moments

In [ ]:
# Load time reference data for challenge moments
d_time_ref = pd.read_csv('final_filtered_test_file_15mins.csv')
d_time_ref.columns = [col.lower().replace(' ', '_').replace('.', '_') for col in d_time_ref.columns]
d_time_ref['time'] = pd.to_datetime(d_time_ref['time']).dt.round('1min')
d_time_ref = d_time_ref[['cf_class_id', 'time']].drop_duplicates()
d_time_ref['challenge_moment'] = True

# Create challenge model dataset
d_model_challenge = d_model.copy()
d_model_challenge['time'] = pd.to_datetime(d_model_challenge['time']).dt.round('15min')
d_model_challenge = pd.merge(d_model_challenge, d_time_ref, on=['cf_class_id', 'time'], how='left')
d_model_challenge['challenge_moment'] = d_model_challenge['challenge_moment'].fillna(0).astype(int)

# 3. Mixed Effects Model with Challenge Moments Interactions and Relative Odds Ratio Model

In [ ]:
# Model with interactions for challenge moments
formula_challenge = """got_help ~ requested_help*challenge_moment +
                     idle*challenge_moment +
                     system_misuse*challenge_moment +
                     struggle*challenge_moment +
                     student_doing_well*challenge_moment"""

model_challenge = smf.mixedlm(formula_challenge, d_model_challenge,
                             groups=d_model_challenge["anon_student_id"],
                             re_formula="~1")
result_challenge = model_challenge.fit()
print(result_challenge.summary())

# Relative Odds Ratio Model (only for idle and challenge)
formula_ror = "got_help ~ challenge_moment * idle"
model_ror = smf.mixedlm(formula_ror, d_model_challenge,
                      groups=d_model_challenge["anon_student_id"],
                      re_formula="~1")
result_ror = model_ror.fit()

# Calculate odds ratios and confidence intervals
coefs = result_ror.params
coef_ci = result_ror.conf_int()
odds_ratios = np.exp(coefs)
lower_CI = np.exp(coef_ci[0])
upper_CI = np.exp(coef_ci[1])

print("Odds Ratios:")
for term, odds_ratio, lower, upper in zip(odds_ratios.index, odds_ratios, lower_CI, upper_CI):
    if term != 'Intercept' and term != 'Group Var':
        print(f"{term}: OR = {odds_ratio:.3f}, 95% CI [{lower:.3f}, {upper:.3f}]")

NameError: name 'd_model_challenge' is not defined

# 4. Relative Odds Ratio Model

In [ ]:
# By-class models (sparse data handling)
def run_class_model(class_data):
    if len(class_data) > 30:  # Minimum sample size check
        try:
            formula = "got_help ~ requested_help + idle + system_misuse + struggle + student_doing_well"
            model = smf.mixedlm(formula, class_data, groups=class_data["anon_student_id"], re_formula="~1")
            result = model.fit()

            # Extract coefficients and standard errors
            coefs = result.params
            se = result.bse

            # Create coefficient dataframe
            coef_df = pd.DataFrame({
                'term': coefs.index,
                'estimate': coefs.values,
                'std.error': se.values,
                'cf_class_id': class_data['cf_class_id'].iloc[0]
            })
            return coef_df
        except:
            return None
    return None

# Run models by class
class_dfs = []
for class_id, class_data in d_model.groupby('cf_class_id'):
    coef_df = run_class_model(class_data)
    if coef_df is not None:
        class_dfs.append(coef_df)

if class_dfs:
    class_coefs = pd.concat(class_dfs, ignore_index=True)
    # Filter out intercepts and high standard errors
    class_coefs = class_coefs[(class_coefs['term'] != 'Intercept') & (class_coefs['std.error'] < 50)]

    # Plot coefficients by class
    plt.figure(figsize=(12, 8))
    ax = sns.pointplot(x='term', y='estimate', hue='cf_class_id', data=class_coefs, dodge=0.5)

    # Add error bars
    for i, row in class_coefs.iterrows():
        plt.errorbar(x=row['term'], y=row['estimate'],
                     yerr=row['std.error'],
                     color=sns.color_palette()[i % 10])  # Cycle through colors

    plt.xticks(rotation=45, ha='right')
    plt.title('Coefficient Estimates by Class')
    plt.tight_layout()
    plt.show()

# 5. Extract random effects

In [ ]:
# Extract random effects
random_effects = result3.random_effects
d_ranef = pd.DataFrame([{'anon_student_id': k, 'intercept': v[0]}
                       for k, v in random_effects.items()])

# Load AFM data
d_afm = d[d['attempt_at_step'] == 1].copy()
d_afm = d_afm.dropna(subset=['kc_model_mat_hia'])
d_afm = d_afm.sort_values(['anon_student_id', 'time'])
d_afm = d_afm[['anon_student_id', 'kc_model_mat_hia', 'outcome']].rename(columns={'kc_model_mat_hia': 'kc'})
d_afm['outcome'] = np.where(d_afm['outcome'] == 'OK', 1, 0)

# Create opportunity counts
d_afm['opportunity'] = d_afm.groupby(['anon_student_id', 'kc']).cumcount() + 1

# Load pre-computed AFM random effects (instead of re-computing)
d_join_iafm = pd.read_pickle('iafm_ranef.rds')  # Assuming we saved this previously
d_join_iafm = pd.DataFrame([{'anon_student_id': k, 'initial_prof': v[0], 'opportunity': v[1]}
                          for k, v in d_join_iafm.items()])

# Join random effects
d_corr = pd.merge(d_ranef, d_join_iafm, on='anon_student_id')

# Compute correlations
corr_initial, p_initial = pearsonr(d_corr['intercept'], d_corr['initial_prof'])
corr_opp, p_opp = pearsonr(d_corr['intercept'], d_corr['opportunity'])

print(f"Correlation with initial proficiency: r = {corr_initial:.3f}, p = {p_initial:.3f}")
print(f"Correlation with opportunity: r = {corr_opp:.3f}, p = {p_opp:.3f}")

# Individual odds ratio calculations with bootstrap
def calculate_odds_ratio(data, var1, var2):
    # Create contingency table
    table = pd.crosstab(data[var1], data[var2])
    # Calculate odds ratio: (a*d)/(b*c)
    try:
        odds_ratio = (table.iloc[1, 1] * table.iloc[0, 0]) / (table.iloc[0, 1] * table.iloc[1, 0])
        return odds_ratio
    except:
        return np.nan

# 6. Bootstrap confidence intervals for requested_help

In [ ]:
# Bootstrap confidence intervals for requested_help
samples_requested = []
for _ in range(1000):
    d_model_resampled = resample(d_model)
    or_value = calculate_odds_ratio(d_model_resampled, 'got_help', 'requested_help')
    samples_requested.append(or_value)

ci_requested = np.percentile(samples_requested, [2.5, 50, 97.5])
print(f"Odds Ratio for requested_help: {ci_requested[1]:.3f} (95% CI: {ci_requested[0]:.3f}-{ci_requested[2]:.3f})")

# Bootstrap for idle
samples_idle = []
for _ in range(1000):
    d_model_resampled = resample(d_model)
    or_value = calculate_odds_ratio(d_model_resampled, 'got_help', 'idle')
    samples_idle.append(or_value)

ci_idle = np.percentile(samples_idle, [2.5, 50, 97.5])
print(f"Odds Ratio for idle: {ci_idle[1]:.3f} (95% CI: {ci_idle[0]:.3f}-{ci_idle[2]:.3f})")

# Bootstrap for system_misuse
samples_misuse = []
for _ in range(1000):
    d_model_resampled = resample(d_model)
    or_value = calculate_odds_ratio(d_model_resampled, 'got_help', 'system_misuse')
    samples_misuse.append(or_value)

ci_misuse = np.percentile(samples_misuse, [2.5, 50, 97.5])
print(f"Odds Ratio for system_misuse: {ci_misuse[1]:.3f} (95% CI: {ci_misuse[0]:.3f}-{ci_misuse[2]:.3f})")

# Bootstrap for struggle
samples_struggle = []
for _ in range(1000):
    d_model_resampled = resample(d_model)
    or_value = calculate_odds_ratio(d_model_resampled, 'got_help', 'struggle')
    samples_struggle.append(or_value)

ci_struggle = np.percentile(samples_struggle, [2.5, 50, 97.5])
print(f"Odds Ratio for struggle: {ci_struggle[1]:.3f} (95% CI: {ci_struggle[0]:.3f}-{ci_struggle[2]:.3f})")

# Bootstrap for student_doing_well
samples_doing_well = []
for _ in range(1000):
    d_model_resampled = resample(d_model)
    or_value = calculate_odds_ratio(d_model_resampled, 'got_help', 'student_doing_well')
    samples_doing_well.append(or_value)

ci_doing_well = np.percentile(samples_doing_well, [2.5, 50, 97.5])
print(f"Odds Ratio for student_doing_well: {ci_doing_well[1]:.3f} (95% CI: {ci_doing_well[0]:.3f}-{ci_doing_well[2]:.3f})")